In [1]:
import os
from PIL import Image
import numpy as np

# Base directory containing the dataset
base_directory = "Zebra_fish_data/Balanced_dataset"

# Subfolder names corresponding to labels
subfolders = {
    1: "images_1",
    2: "images_2",
    3: "images_3",
    4: "images_4"
}

# Initialize lists for data and labels
data = []
labels = []

# Load images and labels from each subfolder
for label, folder_name in subfolders.items():
    folder_path = os.path.join(base_directory, folder_name)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Load image and convert to numpy array
        image = Image.open(file_path)
        image_array = np.array(image)
        
        # Append image and label to respective lists
        data.append(image_array)
        labels.append(label)

# Confirm loaded data
print(f"Loaded {len(data)} images into 'data' list.")


Loaded 166 images into 'data' list.


In [4]:
resized_data = [Image.fromarray(image).resize((224, 224)) if isinstance(image, np.ndarray) else image.resize((224, 224)) for image in data]


In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import timm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np

seed = 111

# Configure the ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.20,
    height_shift_range=0.20,
    zoom_range=0.10,
    horizontal_flip=False,
    fill_mode='nearest',
)

# Function to apply data augmentation to a batch
def apply_data_augmentation(batch_data):
    batch_data_array = np.array(batch_data)
    augmented_batch_data = []
    for image in batch_data_array:
        augmented_image = train_datagen.random_transform(image)
        augmented_batch_data.append(augmented_image)
    return augmented_batch_data

# Feature extractor function (you might want to adjust this based on the specific model you are using)
from transformers import AutoFeatureExtractor
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/deit-small-distilled-patch16-224")

# Split the data into 80% learning and 20% test data
X_learn, test_data, y_learn, test_labels = train_test_split(
    data,
    labels,
    test_size=0.2,
    stratify=labels,
    random_state=seed
)

# Split the learning data into training and validation sets (90% train, 10% validation)
train_data, val_data, train_labels, val_labels = train_test_split(
    X_learn,
    y_learn,
    test_size=0.1,
    stratify=y_learn,
    random_state=seed
)

# Create DataLoaders for training, validation, and testing
train_loader = DataLoader(list(zip(train_data, train_labels)), batch_size=25, shuffle=True)
val_loader = DataLoader(list(zip(val_data, val_labels)), batch_size=25, shuffle=True)
test_loader = DataLoader(list(zip(test_data, test_labels)), batch_size=25, shuffle=True)

# Load the pre-trained DeiT model with 224x224 resolution
model = timm.create_model('deit_small_distilled_patch16_224', pretrained=True)

# Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.00008)
criterion = CrossEntropyLoss()

# Define the number of epochs
num_epochs = 45

# Lists to track metrics
training_loss_list = []
training_acc_list = []
val_acc_list = []

c:\Users\9andy\anaconda3\lib\site-packages\transformers\models\deit\feature_extraction_deit.py:28: FutureWarning: The class DeiTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DeiTImageProcessor instead.
  warnings.warn(


In [ ]:
# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')  # Print the current epoch number
    
    # Initialize metrics for training
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    # Training phase
    model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        inputs = apply_data_augmentation(inputs)  # Apply data augmentation to the input batch
        optimizer.zero_grad()  # Reset gradients from the previous step
        
        # Preprocess inputs and perform forward pass
        batch = feature_extractor(list(inputs), return_tensors="pt", do_normalize=True)
        pixel_values = batch['pixel_values']  # Extract the tensor for the model input

        outputs = model(pixel_values)
        
        # Calculate loss and perform backward pass
        loss = criterion(outputs, labels)
        loss.backward()  # Compute gradients
        optimizer.step()  # Update model weights
        
        # Accumulate loss and accuracy metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    # Calculate average training loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_predictions
    
    # Store training metrics for later analysis
    training_loss_list.append(epoch_loss)
    training_acc_list.append(epoch_accuracy)

    print(f"Train Acc: {epoch_accuracy}")

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_correct_predictions = 0
    val_total_predictions = 0
    
    with torch.no_grad():  # Disable gradient computation for validation
        for inputs, labels in val_loader:
            batch = feature_extractor(list(inputs), return_tensors="pt", do_normalize=True)
            pixel_values = batch['pixel_values']
            outputs = model(pixel_values)
            
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)
    
    # Calculate validation accuracy for the epoch
    val_accuracy = val_correct_predictions / val_total_predictions
    print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

    val_acc_list.append(val_accuracy)


# Log metrics and test labels
log_directory = f"Models/Transformer"
os.makedirs(log_directory, exist_ok=True)

# Save the model at the end of training (after the final epoch)
final_model_path = os.path.join(log_directory, 'trans_model.pth')
torch.save(model.state_dict(), final_model_path)
print(f'Model saved to {final_model_path}')

# Save metrics
metrics_path = os.path.join(log_directory, 'trans_metrics.txt')
with open(metrics_path, 'w') as f:
    f.write('Training Loss List:\n')
    f.write(str(training_loss_list) + '\n\n')
    f.write('Training Accuracy List:\n')
    f.write(str(training_acc_list) + '\n\n')
    f.write('Validation Accuracy List:\n')
    f.write(str(val_acc_list) + '\n\n')

# Save test labels
test_labels_path = os.path.join(log_directory, 'test_labels.txt')
test_indices = test_labels.index.tolist()
with open(test_labels_path, 'w') as f:
    f.write('Test Labels with Row Indices:\n')
    for index, label in zip(test_indices, test_labels):
        f.write(f'Row {index}: Label {label}\n')


Epoch 1/45
Train Acc: 0.1864406779661017
Validation Accuracy: 35.71%
Epoch 2/45
Train Acc: 0.4491525423728814
Validation Accuracy: 57.14%
Epoch 3/45
Train Acc: 0.5254237288135594
Validation Accuracy: 64.29%
Epoch 4/45
Train Acc: 0.6186440677966102
Validation Accuracy: 71.43%
Epoch 5/45
Train Acc: 0.711864406779661
Validation Accuracy: 42.86%
Epoch 6/45


In [3]:
import os
import json
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

# Data augmentation function
def preprocess_function(image):
    # Adjust contrast of the image
    image = tf.image.adjust_contrast(image, 1.2)
    return image

# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    horizontal_flip=True,  # Randomly flip images horizontally
    rotation_range=20,  # Randomly rotate images
    zoom_range=0.2,  # Randomly zoom images
    fill_mode='nearest',  # Fill in missing pixels after transformations
    preprocessing_function=preprocess_function  # Custom preprocessing function
)

# Load the pre-trained VGG16 model
# Include only convolutional base (no fully connected layers) and use the specified input shape
input_shape = (224, 224, 3)  # Specify the input shape (224x224 with 3 color channels)
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze all layers in the pre-trained model
for layer in vgg16.layers:
    layer.trainable = False

# Add custom classification head
x = Flatten()(vgg16.output)  # Flatten the feature map into a 1D vector
x = Dense(512, activation='relu', kernel_regularizer=l1(0.001))(x)  # Fully connected layer with L1 regularization
x = Dropout(0.5)(x)  # Dropout to prevent overfitting
predictions = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Define the complete model
model = Model(inputs=vgg16.input, outputs=predictions)

# Print a summary of the model
model.summary()

# Normalize the data and convert to NumPy arrays
X_train = np.array([np.array(image) for image in train_data], dtype=np.float32) / 255.0
X_val = np.array([np.array(image) for image in val_data], dtype=np.float32) / 255.0
X_test = np.array([np.array(image) for image in test_data], dtype=np.float32) / 255.0

# Preprocess input data for VGG16 (standardize based on ImageNet)
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

# Convert labels to NumPy arrays
y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

# Verify the shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Create a training data generator with data augmentation
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Optimizer with specified learning rate
    loss='binary_crossentropy',  # Binary cross-entropy loss for binary classification
    metrics=['accuracy']  # Track accuracy during training
)

# Train the model
history = model.fit(
    train_generator,  # Use the augmented data generator for training
    epochs=num_epochs,  # Train for 45 epochs (adjust as needed)
    validation_data=(X_val, y_val)  # Use the validation set for evaluation
)

# Save metrics after training
save_directory = f"Models/VGG16"
os.makedirs(save_directory, exist_ok=True)

metrics = {
    "train_loss": history.history['loss'],  # Training loss for each epoch
    "train_accuracy": history.history['accuracy'],  # Training accuracy for each epoch
    "val_accuracy": history.history['val_accuracy'],  # Validation accuracy for each epoch
}

metrics_path = os.path.join(save_directory, "vgg_16_metrics.json")
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=4)
print(f"Metrics saved to {metrics_path}")

# Save the trained model
model_save_path = os.path.join(save_directory, "vgg_16_model.h5")
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

InvalidArgumentError:  Input to reshape is a tensor with 6553600 values, but the requested shape requires a multiple of 25088
	 [[node model/flatten/Reshape
 (defined at c:\Users\9andy\anaconda3\lib\site-packages\keras\layers\core\flatten.py:96)
]] [Op:__inference_train_function_1575]

Errors may have originated from an input operation.
Input Source operations connected to node model/flatten/Reshape:
In[0] model/block5_pool/MaxPool (defined at c:\Users\9andy\anaconda3\lib\site-packages\keras\layers\pooling.py:357)	
In[1] model/flatten/Const (defined at c:\Users\9andy\anaconda3\lib\site-packages\keras\layers\core\flatten.py:91)

Operation defined at: (most recent call last)
>>>   File "c:\Users\9andy\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "c:\Users\9andy\anaconda3\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "c:\Users\9andy\anaconda3\lib\site-packages\ipykernel_l